#### Version 1 [1/7/2025]:
    - This code generates consolidated report with leads (Drawdown, Opportunities and Calls) 

#### Pre-requisite files:
    - Salesforce extracts (drawdown, opportunities and calls)

---------------------------------------------------------------------------------------------------------------------------

#### Import Modules

In [4]:
import os
import numpy as np # numerical calc package
import pandas as pd # holds data
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # pretty plotting
import warnings
import time
import datetime as dt
warnings.filterwarnings('ignore')

#### Date Filters

In [5]:
#Edit to change dates and year
input_year = 2025
input_month = 2
input_day = 7     #friday  
input_date = dt.date(input_year,input_month,input_day)
date_dir1 = dt.date(input_year,input_month,(input_day-1))

date = input_date.strftime('%Y%m%d')
year = input_date.strftime('%Y')
date_dir = date_dir1.strftime('%Y%m%d')

print(date)
print(year)
print(input_date)
print(date_dir)
start_time = time.time()

20250207
2025
2025-02-07
20250206


#### Loading files

In [7]:
#Reading datasets from excel files
dd_acct = pd.read_excel(f"//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/Leads/{date_dir}/Drawdown Tracker - {date_dir}.xlsx")
dd_leads = pd.read_excel(f"//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/Leads/{date_dir}/Drawdown Tracker Leads - {date_dir}.xlsx")

# call_acct = pd.read_excel(f"D:/OneDrive - Security Bank Corporation/Desktop/Projects/WBS Weekly OBs/RM Workbench Prep/leads/Call Reports on Accounts.xlsx")
# call_leads = pd.read_excel(f"D:/OneDrive - Security Bank Corporation/Desktop/Projects/WBS Weekly OBs/RM Workbench Prep/leads/Call Reports on Leads.xlsx")


opp_acct = pd.read_excel(f"//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/Leads/{date_dir}/Opportunities.xlsx")
opp_leads = pd.read_excel(f"//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/Leads/{date_dir}/Opportunities and Leads.xlsx")

wbs_roster = pd.read_excel(f"//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/RM Weekly/RM Weekly {date_dir}/WBS Roster as of {date_dir} v2.xlsx", sheet_name="KPIs")

#Rename columns under Leads
#Drawdown
dd_rename_col = {'Lead: Company':'Account: Account Name'
             ,'Lead: Lead Owner':'Account: Account Owner'
             ,'Lead: Customer Group':'Account: Parent Account'
             ,'Lead: Team Center':'Account: Owner Team Center (Account)'}
dd_leads1 = dd_leads.rename(columns=dd_rename_col) 

#Calls
# call_rename_col = {'Company Name (Lead)':'Account Name: Account Name'
#              ,'Lead Name: Lead Owner':'Account Owner'
#              ,'Lead Name: Team Center':'Team Center (Account Owner)'}
# call_leads1 = call_leads.rename(columns=call_rename_col) 

#Opportunities
opp_rename_col = {"Opportunity Owner: Full Name":"Opportunity Owner"
             ,"Company (Lead)":"Account Name"
             ,"Lead Owner: Full Name":"Account Owner"
             ,"Product Name: Product Name":"Product Name"
             ,"ADB (In '000) (converted)":"ADB (In ‘000) (converted)"}
opp_leads1 = opp_leads.rename(columns=opp_rename_col) 

In [4]:
##For checking values only
# # print(dd_acct.shape)
# print(dd_leads.columns)
# print(dd_leads1.columns)
# print('----------------------------------------------------------------------')
# print(call_leads.columns)
# print(call_leads1.columns)
# print('----------------------------------------------------------------------')
# print(opp_leads.columns)
# print(opp_acct.columns)
# print(opp_leads1.columns)

#### Tranform data

In [8]:
#Add columns - Leads and Non-Lead
dd_acct['Lead/Account'] = 'Account'
dd_leads1['Lead/Account'] = 'Lead'

# call_acct['Lead/Account'] = 'Account'
# call_leads1['Lead/Account'] = 'Lead'

opp_acct['Lead/Account'] = 'Account'
opp_leads1['Lead/Account'] = 'Lead'

In [9]:
opp_leads1.count()

Opportunity Owner                                      610
Account Name                                           610
Account Owner                                          610
Lead ID                                                610
Team Center                                            610
Nature of Business                                     610
Created Date                                           610
Lead Source                                            610
Lead Status                                            610
Opportunity Name                                       610
Owner Team Center (Opportunity)                        610
Product                                                349
ADB (In ‘000) (converted)                              610
Volume (In '000) (converted)                           610
Expected Revenue Total (In '000) (converted)           610
Expected Revenue Current Year (In '000) (converted)    610
Stage                                                  6

In [10]:
#Appending into 1 list

#Drawdown
dd_list = [dd_acct,dd_leads1]
dd_append = pd.concat(dd_list, axis=0)

#Call
# call_list = [call_acct,call_leads1]
# call_append = pd.concat(call_list, axis=0)

#Opportunities
opp_list = [opp_acct,opp_leads1]
opp_append = pd.concat(opp_list, axis=0)

In [11]:
#Re-arrange columns
# call_reorder = call_append.reindex(['Call Report: Call Report Name', 'Account Name: Account Name'
#                                ,'Account Name: BBN','Account Owner','Account Name: AO Code'
#                                ,'Call Report: Created Date','Date of Call','Attendees from Companies'
#                                ,'Account Name: Business Segment Type','Account Name: Client Segment Type'
#                                ,'Account Name: Owner Segment Type (Account)','Call Report: Created By'
#                                ,'Call Report: Owner Name','Team Center (Call Report Owner)'
#                                ,'Team Center (Account Owner)','Purpose of Call'
#                                ,'Venue of Meeting','Generates New Business','SBC Participants'
#                                ,'Highlight','Target Date','Details','Lead/Account'], axis=1)

opp_reorder = opp_append.reindex(['Opportunity Owner','Team Center','Opportunity Name'
                                   ,'BBN','Account Name','Account Owner','Client Segment Type'
                                   ,'Business Segment Type','Company (Lead)','Type','Product Name'
                                   ,'Product Family','Stage',"Volume (In '000) (converted)"
                                   ,'ADB (In ‘000) (converted)',"Expected Revenue Current Year (In '000) (converted)"
                                   ,"Expected Revenue Total (In '000) (converted)",'Target Close Date'
                                   ,'Created Date','Opportunity ID','Won Reason','Loss Reason'
                                   ,'Lead/Account'], axis=1)

In [13]:
# #Identify WBS Roster BBNs
# wbs_roster['Roster_BBNs'] = 1

# #Merging these columns to drawdowns and opportunities
# wbs_roster2 = wbs_roster[['BBN','Client Group','Pareto/Client Segmentation','STATUS','Roster_BBNs']]

# dd_append['Client Group','Pareto/Client Segmentation','STATUS','Roster_BBNs'] = dd_append['Account: BBN'].map(wbs_roster.set_index('BBN')['Roster_BBNs'])
# opp_reorder['Client Group','Pareto/Client Segmentation','STATUS','Roster_BBNs'] = opp_reorder['BBN'].map(wbs_roster.set_index('BBN')['Roster_BBNs'])

In [12]:
# Identify WBS Roster BBNs
wbs_roster['Roster_BBNs'] = 1

# Selecting necessary columns from wbs_roster
wbs_roster2 = wbs_roster[['BBN', 'Client Group', 'Pareto/Client Segmentation', 'STATUS','Business_Segment','Roster_BBNs']]

# Merging with dd_append
dd_append = dd_append.merge(wbs_roster2, left_on='Account: BBN', right_on='BBN', how='left')

# Merging with opp_reorder
opp_reorder = opp_reorder.merge(wbs_roster2, on='BBN', how='left')

In [13]:
dd_append.count()

Drawdown Tracker: Owner Name                 3867
Team Center                                  3867
Drawdown Tracker: Drawdown Tracker Number    3867
Account: Account Name                        3867
Account: Parent Account                      2676
Account: Account Owner                       3867
Account: Owner Team Center (Account)         3867
Drawdown Tracker: Currency                   3867
Amount (In '000) Currency                    3867
Amount (In '000)                             3867
Tenor (In days)                              3867
Rate (In %)                                  3857
Facility Type                                3867
Other Facility Type                           560
Remarks                                      1149
Status                                       3865
Probability of Success                       3633
Target Booking Date                          3867
Deal Won Date                                3559
Lost Reason                                   108


In [14]:
print(dd_append.info())
print('--------------------------------------------------------------------')
# print(call_reorder.info())
print('--------------------------------------------------------------------')
print(opp_reorder.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3867 entries, 0 to 3866
Data columns (total 31 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Drawdown Tracker: Owner Name               3867 non-null   object        
 1   Team Center                                3867 non-null   object        
 2   Drawdown Tracker: Drawdown Tracker Number  3867 non-null   object        
 3   Account: Account Name                      3867 non-null   object        
 4   Account: Parent Account                    2676 non-null   object        
 5   Account: Account Owner                     3867 non-null   object        
 6   Account: Owner Team Center (Account)       3867 non-null   object        
 7   Drawdown Tracker: Currency                 3867 non-null   object        
 8   Amount (In '000) Currency                  3867 non-null   object        
 9   Amount (In '000)   

#### Exporting to Excel File

In [15]:
#Drawdown
writer = pd.ExcelWriter(f'//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/RM Weekly/RM Weekly {date_dir}/Drawdown Tracker v2.xlsx')
dd_append.to_excel(writer, 
                    sheet_name='Drawdown Tracker', 
                    index=False)
writer.close()

In [12]:
# #Calls
# writer = pd.ExcelWriter(f'//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/RM Weekly/RM Weekly {date_dir}/SFCall Report_drop1.2 v3.xlsx')
# call_reorder.to_excel(writer, 
#                     sheet_name='Call Report', 
#                     index=False)
# writer.close()

In [16]:
#Calls
writer = pd.ExcelWriter(f'//P1vm-dwhfsvr01/BID/MRT/LENDING/REPORTS/3. Monthly Reports/COMMERCIAL/RM Workbench/01 Input/RM Weekly/RM Weekly {date_dir}/Opportunities v2.xlsx')
opp_reorder.to_excel(writer, 
                    sheet_name='Opportunities', 
                    index=False)
writer.close()

In [17]:
end_time = time.time() 
minutes = (end_time - start_time)/60
print("Execution time: "+str(minutes) +" mins")

Execution time: 38.55765359004339 mins
